In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli

In [ ]:

!pip install sentencepiece

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=3000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':1})

In [ ]:
# from langchain import PromptTemplate,  LLMChain

# template = """
#               Write a concise summary of the following debate text delimited by triple backquotes.
#               Return your response in 3 bullet points which covers the key points of the text.

#               ```{text}```
#               BULLET POINT SUMMARY:
#            """

# prompt = PromptTemplate(template=template, input_variables=["text"])

# llm_chain = LLMChain(prompt=prompt, llm=llm)

# text = """
# BIDEN: How are you doing, man?

# TRUMP: How are you doing (ph)?

# BIDEN: I'm well.

# WALLACE: Gentlemen, a lot of people have been waiting for this night. So let's get going. Our first subject is the Supreme Court.

# President Trump, you nominated Amy Coney Barrett over the weekend to succeed the late Ruth Bader Ginsburg on the court.

# You say the Constitution is clear about your obligation and the Senate's to consider a nominee to the court. Vice President Biden, you say that this is an effort by the president and Republicans to jam through an appointment and what you call an abuse of power.

# My first question to both of you tonight, why are you right and make the argument you make, and your opponent wrong? And where do you think a Justice Barrett would take the court?

# President Trump, on the first segment you go first. Two minutes.

# TRUMP: Thank you very much, Chris. I will tell you very simply we won the election. Elections have consequences. We have the Senate, we have the White House and we have a phenomenal nominee respected by all; top, top academic. Good in every way. Good in every way.

# In fact, some of her biggest endorsers are very liberal people from Notre Dame and other places. So I think she's going to be fantastic. We have plenty of time. Even if we did it after the election itself, I have a lot of time after the election, as you know.

# So I think that she will be outstanding. She's going to be as good as anybody that has served on that court. We really feel that. We have a professor at Notre Dame, highly respected by all; said she's the single greatest student he's ever had. He's been a professor for a long time at a great school.

# And we just -- we won the election and therefore we have the right to choose her, and very few people knowingly would say otherwise.

# And by the way, the Democrats, they wouldn't even think about not doing it. If they had -- the only difference is they'd try and do it faster. There's no way they would give it up. They had Merrick Garland, but the problem is they didn't have the election. So they were stopped.

# And probably that would happen in reverse also. Definitely it would happen in reverse. So we won the election and we have the right to do it, Chris.

# WALLACE: President Trump, thank you. Same question to you, Vice President Biden. You have two minutes.

# BIDEN: Well, first of all, thank you for doing this and looking forward to this, Mr. President.

# TRUMP: Thank you, Joe.
# """

# print(llm_chain.run(text))

In [ ]:
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import requests
# url = "https://www.gutenberg.org/cache/epub/71224/pg71224.txt"
# response = requests.get(url)
# if response.status_code == 200:
#     data = response.text
url = "/content/presidential_debate_transcript.txt"
with open(url, 'r') as file:
     data = file.read()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=500, chunk_overlap=200)
docs = text_splitter.create_documents([data])

print (f"You now have {len(docs)} docs intead of 1 piece of text")


You now have 325 docs intead of 1 piece of text


In [ ]:
map_prompt = """
Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

In [ ]:
combine_prompt = """

 Write a concise summary in bullet points of the following text delimited by triple backquotes.

 ```{text}```
 BULLET POINT SUMMARY:
 """
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [ ]:
summary_chain = load_summarize_chain(llm=llm,
  chain_type='map_reduce',

 map_prompt=combine_prompt_template,
  )


In [ ]:
output = summary_chain.run(docs)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (75024 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


ValueError: Input length of input_ids is 3174, but `max_length` is set to 3000. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [ ]:
print(output)

In [ ]:
torch.cuda.empty_cache()